In [1]:
# imports
from owlready2 import *
import types
import gzip
import pandas as pd
from rdflib import *
import owlrl

In [2]:
# owlready2 functions

# get Classes
def getClasses(onto):        
    return onto.classes()

# get Data Properties    
def getDataProperties(onto):        
    return onto.data_properties()

# get Objecct Properties       
def getObjectProperties(onto):        
    return onto.object_properties()

# get Individuals   
def getIndividuals(onto):        
    return onto.individuals()

# load ontology
def loadOntology(urionto): 
    onto = get_ontology(urionto).load()
    return onto

# function to return labels for GO entities
def my_rendering(entity):
    return entity.label.first()

In [4]:
# Load mappings from STRING ID to UNIPROT Protein ID for lookups / this code is reworked from OPA2Vec repository

org_id = '9606' # Change to 9606 for Human

# preprocess GO annotations
mapping = {}
source = {'4932': 'SGD_ID', '9606': 'Ensembl_UniProt_AC'} # mapping source

with gzip.open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/STRINGDB/{org_id}.protein.aliases.v11.5.txt.gz', 'rt', encoding='utf-8') as f:
    next(f) # Skip header
    for line in f:
        string_id, p_id, sources = line.strip().split('\t')
        if source[org_id] not in sources.split():
            continue
        if p_id not in mapping:
            mapping[p_id] = set()
        mapping[p_id].add(string_id)
print('Loaded mappings', len(mapping))

Loaded mappings 83972


In [8]:
# load GO functions for each protein with mapping # code is reworked from OPA2Vec Repository
gaf_files = {'4932': 'sgd.gaf.gz', '9606': 'goa_human.gaf.gz'}
annotations = set()
with gzip.open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/GeneAnnotationFile/{gaf_files[org_id]}', 'rt', encoding='utf-8') as f:
    for line in f:
        if line.startswith('!'): # Skip header
            continue
        it = line.strip().split('\t')
        p_id = it[1]
        go_id = it[4]
        relation_id = it[3]
        object_name = it[9]
        if it[6] == 'IEA' or it[6] == 'ND': # Ignore predicted or no data annotations
            continue
        if p_id not in mapping: # Not in StringDB
            continue
        s_ids = mapping[p_id]
        for s_id in s_ids:
            annotations.add((s_id, go_id, object_name, relation_id))
print('Number of annotations:', len(annotations))
#print('Number of unique proteins:', )

# Save annotations
with open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/STRINGDB/{org_id}.proteinfunctions.txt', 'w') as f:
    for s_id, go_id, object_name, relation_id in annotations:
        f.write(f'{s_id}\t{s_id}\t{go_id}\t{object_name}\t{relation_id}\n')

Number of annotations: 209463


In [9]:
with gzip.open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/GeneAnnotationFile/{gaf_files[org_id]}', 'rt', encoding='utf-8') as f:
    for line in f:
        if line.startswith('!'): # Skip header
            continue
        it = line.strip().split('\t')
        print(it)
        break

['UniProtKB', 'A0A024RBG1', 'NUDT4B', 'enables', 'GO:0003723', 'GO_REF:0000043', 'IEA', 'UniProtKB-KW:KW-0694', 'F', 'Diphosphoinositol polyphosphate phosphohydrolase NUDT4B', 'NUDT4B', 'protein', 'taxon:9606', '20210612', 'UniProt']


In [11]:
# generate protein-proten interactions

interactions = {}
data = []
with gzip.open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/STRINGDB/{org_id}.protein.links.v11.5.txt.gz', 'rt') as f:
    next(f) # Skip header
    for line in f:
        p1, p2, score = line.strip().split()
        if float(score) < 700: # Filter high confidence interactions
            continue
        if p1 not in interactions:
            interactions[p1] = set()
        if p2 not in interactions:
            interactions[p2] = set()
        if p2 not in interactions[p1]:
            interactions[p1].add(p2)
            interactions[p2].add(p1)
            data.append((p1, p2))

print('Total number of interactions:', len(data))
print('Total number of proteins:', len(interactions.keys()))

Total number of interactions: 5969249
Total number of proteins: 19385


In [12]:
len(data)

5969249

In [ ]:
# generate negative protein-protein interactions # OPA2Vec methodology and code
import random
proteins =set ()
negatives = []
for (p1,p2) in data:
        proteins.add(p1)
        proteins.add(p2)
while len(negatives)< (len(data)/3):
        s = random.sample(proteins, 2)
        prot1= s[0]
        prot2= s[1]
        if (prot1,prot2) in negatives or (prot2,prot1) in negatives :
                 continue
        if prot1 not in interactions[prot2]:
                 negatives.append((prot1, prot2))
                 
# Save negatives data
with open(f'/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/STRINGDB/{org_id}.negative_interactions.txt', 'w') as f:
    for p1 , p2 in negatives:
        f.write(f'{p1}\t{p2}\t\n')

In [ ]:
# read protein links
df_p_pairs = pd.read_csv('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/9606.protein.links.v11.5.txt', sep='\s', engine='python')

# positive pairs: all scores above 0
df_positive_pairs = df_p_pairs[df_p_pairs['combined_score'] > 0]
# text preprocessing
df_positive_pairs.loc[:,'protein1'] = df_positive_pairs.loc[:,'protein1'].str.replace('.','_')
df_positive_pairs.loc[:,'protein2'] = df_positive_pairs.loc[:,'protein2'].str.replace('.','_')

In [12]:
# inspect a protein
df_positive_pairs.loc[df_positive_pairs['protein1'] == '9606_ENSP00000393963']

,protein1,protein2,combined_score
9676792,9606_ENSP00000393963,9606_ENSP00000265148,854
9676794,9606_ENSP00000393963,9606_ENSP00000355090,710
9676807,9606_ENSP00000393963,9606_ENSP00000274026,906
9676812,9606_ENSP00000393963,9606_ENSP00000404833,708
9676814,9606_ENSP00000393963,9606_ENSP00000442068,898
...,...,...,...
9677969,9606_ENSP00000393963,9606_ENSP00000363794,784
9677975,9606_ENSP00000393963,9606_ENSP00000480987,808
9677989,9606_ENSP00000393963,9606_ENSP00000362146,946
9678014,9606_ENSP00000393963,9606_ENSP00000357643,833


In [ ]:
# number of unique proteins
len(pd.unique(df_positive_pairs[['protein1','protein2']].values.ravel('K')))

16795

In [ ]:
# make dataframe for annotations
df_protein_functions = pd.DataFrame(annotations).rename(columns={ 0:'Protein_String_ID', 1:'GO_Term', 2:'Protein_Label', 3:"Object_Property"})
# replace GO:0030314 with GO_0030314
df_protein_functions['GO_Term'] = df_protein_functions['GO_Term'].replace(':','_', regex=True)
df_protein_functions['Protein_String_ID'] = df_protein_functions['Protein_String_ID'].str.replace('.','_', regex=True)
df_protein_functions['Object_Property'] = df_protein_functions['Object_Property'].astype(str).str.replace('NOT|','_not_', regex=False)

In [16]:
# inspect protein functions from Gene Annotation Files
df_protein_functions

,Protein_String_ID,GO_Term,Protein_Label,Object_Property
0,9606_ENSP00000323421,GO_0008278,Structural maintenance of chromosomes protein 1A,part_of
1,9606_ENSP00000254436,GO_0010508,E3 ubiquitin-protein ligase TRIM21,involved_in
2,9606_ENSP00000255078,GO_0005524,DNA-binding protein SMUBP-2,enables
3,9606_ENSP00000294740,GO_0005654,Zinc finger protein 281,located_in
4,9606_ENSP00000370473,GO_0031994,Insulin-like growth factor-binding protein 3,enables
...,...,...,...,...
209458,9606_ENSP00000335203,GO_0042030,"ATPase inhibitor, mitochondrial",enables
209459,9606_ENSP00000237596,GO_0072208,Polycystin-2,involved_in
209460,9606_ENSP00000265857,GO_0071816,Golgi to ER traffic protein 4 homolog,involved_in
209461,9606_ENSP00000358921,GO_0005813,Alpha-centractin,located_in


In [17]:
# inspect all NOT relations 
for i in df_protein_functions['Object_Property']:
        if 'not' in i :
            print(i)

_not_enables
_not_involved_in
_not_enables
_not_involved_in
_not_involved_in
_not_involved_in
_not_enables
_not_involved_in
_not_enables
_not_located_in
_not_enables
_not_involved_in
_not_enables
_not_involved_in
_not_involved_in
_not_involved_in
_not_involved_in
_not_involved_in
_not_enables
_not_enables
_not_involved_in
_not_involved_in
_not_involved_in
_not_involved_in
_not_enables
_not_enables
_not_involved_in
_not_enables
_not_enables
_not_enables
_not_involved_in
_not_located_in
_not_enables
_not_involved_in
_not_enables
_not_enables
_not_enables
_not_colocalizes_with
_not_enables
_not_involved_in
_not_involved_in
_not_involved_in
_not_involved_in
_not_enables
_not_contributes_to
_not_enables
_not_enables
_not_involved_in
_not_involved_in
_not_enables
_not_involved_in
_not_enables
_not_involved_in
_not_enables
_not_located_in
_not_enables
_not_enables
_not_involved_in
_not_involved_in
_not_enables
_not_enables
_not_located_in
_not_involved_in
_not_involved_in
_not_involved_in
_no

Ontology Engineering

In [ ]:
# load relations ontology
urionto_RO = '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/ro.xml'
onto_RO = get_ontology(urionto_RO).load()
# load GO ontology
urionto_GO = '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_data/go.owl'
onto_GO = get_ontology(urionto_GO).load() 

In [ ]:
# get list of object properties from relations ontology
properties_RO = []
for prop_ro in getObjectProperties(onto_RO):
    properties_RO.append(prop_ro.iri)

# get list of object properties from GO
properties_GO = []
for prop_go in getObjectProperties(onto_GO):
    properties_GO.append(prop_go.iri)

In [20]:
# string match function for object property alignment
g1 = Graph()
for prop_go in properties_GO:
    for prop_ro in properties_RO:
        if prop_go == prop_ro:
            g1.add((URIRef(prop_go), OWL.equivalentProperty, URIRef(prop_ro)))
        else:
            continue    

#serialize
g1.serialize( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/property_matches.xml',format = 'xml')        

In [21]:
# parse relations ontology, gene ontology and equivalent properties to same graph
g_all = Graph()
g_all.parse( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/go.owl')
g_all.parse( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/ro.xml' )
g_all.parse('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/property_matches.xml')
g_all.serialize('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/GO_RO_processed.xml',format = 'xml')

In [22]:
# load ontology with GO and Relations Ontology parsed inlcuding equivalent properties
uriOnto = '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/GO_RO_processed.xml'
onto = loadOntology(uriOnto)
# get namespace
onto_GO = onto.get_namespace("http://purl.obolibrary.org/obo/")

In [23]:
# get all properties
properties_all = {}
for prop_go in getObjectProperties(onto):
    if len(prop_go.label) > 0:
        properties_all[prop_go.label[0]] = prop_go

# change key names replaces whitespace with an underscore
for key,value in properties_all.items():
    properties_all = { k.replace(' ', '_'): v for k, v in properties_all.items() }

In [24]:
# modify protein functions dataframe to avoid duplicate equivalent classes
dict_prots = {}
for go_term, property_name in zip(df_protein_functions.GO_Term,df_protein_functions.Object_Property):
    property_name = property_name.replace(' ', '_')
    dict_prots[go_term] = property_name

#for key, value in dict_prots.items():
#    dict_prots[key] = value.replace(' ','_').replace('NOT|','_not_')

In [25]:
# map equivalences and modify GO with RDFLib & Owlready2
G_equivalences = Graph()
n_space = Namespace("http://purl.obolibrary.org/obo/")

with onto_GO:
    # add new Protein class in correct namespace
    class Protein(Thing):
        pass
    
    # make new Protein Class subclass of Thing
    superClass = [Thing]

    # create new equivalent protein classes (i.e. obo.protein_involved_in_GO_0003132 owl:equivalentClass obo.Protein)
    for cls in getClasses(onto):
        if str(cls)[4:] in dict_prots.keys():
            class_name = 'protein_' + dict_prots[str(cls)[4:]] + '_' + str(cls)[4:]      
            
            # make new protein class
            new_protein_class = types.new_class(class_name, tuple(superClass))
            
            #use RDFLib instead of OWLREADY2 (slow) to write out equivalent classes
            G_equivalences.add((URIRef(n_space + class_name), OWL.equivalentClass, URIRef(n_space + 'Protein')))
            
            # lookup relevant property, create new protein and generate existential restriction (retrived relation, some, go_term)
            lookup_go = dict_prots[str(cls)[4:]] 
            if lookup_go in properties_all.keys():
                new_protein_class.is_a.append(properties_all[lookup_go].some(cls)) 
    
    # serialize equivalences      
    G_equivalences.serialize('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/equivalent_class.xml', format = 'xml')

In [26]:
#save modified GO with Owlready2
onto.save('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/processed_ontology_human.xml', format = "rdfxml")

In [27]:
# load ontology
uriOnto = '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/processed_ontology_human.xml'
onto = loadOntology(uriOnto)

# get namespace
onto_GO = onto.get_namespace("http://purl.obolibrary.org/obo/")

In [ ]:
# check restrictions work
onto_GO.protein_involved_in_GO_0070370.is_a

In [29]:
# make dictionary of classes with URI
GO_dictionary = {}
with onto_GO:
    for cls in getClasses(onto):
        GO_dictionary[str(cls)] = cls.iri

In [30]:
# initialize empty graph
g = Graph()

t0 = time.time()
n_space = Namespace("http://purl.obolibrary.org/obo/")

# associate proteins to new ontology via correct object property 
for index,row in df_protein_functions.iterrows():
    class_name = 'protein_' + str(row['Object_Property']) + '_' + str(row['GO_Term'])
    protein_label = row['Protein_Label']
    protein_string_id = n_space + row['Protein_String_ID']
    # add triples
    g.add((URIRef(protein_string_id), RDF.type, URIRef(n_space + class_name)))
    g.add((URIRef(protein_string_id), RDFS.label, Literal(protein_label)))
    g.add((URIRef(protein_string_id), RDF.type, OWL.NamedIndividual))

# save graph
g.serialize(destination='/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/protein_instances_rdf.xml', format='xml')

# duration
t1 = time.time()
total = t1-t0
print(total)

64.8437180519104


In [32]:
# initialize empty graph
g = Graph()

t0 = time.time()
n_space = Namespace("http://purl.obolibrary.org/obo/")

# generate protein instances for GO terms. Because of prior equivalence mapping, proteins will also 
# be instances of equivalent classes (i.e. p1 rdf:type protein_located_in_GO_0000323, where obo.protein_located_in_GO_000323 owl:equivalentClass obo.Protein)
for index,row in df_protein_functions.iterrows():
    # get go_term affiliated with relevant protein
    key = "_" + row['GO_Term']
    # list comprehension to match GO term against dictionary
    key_name = [k for k in GO_dictionary.keys() if key in k]
    if len(key_name) > 0:
        go_term = GO_dictionary[key_name[0]]
        protein_string_id = n_space + row['Protein_String_ID']
        protein_label = row['Protein_Label']
        # add triple
        g.add((URIRef(protein_string_id), RDF.type, URIRef(go_term)))
        g.add((URIRef(protein_string_id), RDFS.label, Literal(protein_label)))
        g.add((URIRef(protein_string_id), RDF.type, OWL.NamedIndividual))

# save graph
g.serialize(destination='/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/protein_instances_rdf.xml', format='xml')

t1 = time.time()
total = t1-t0
print(total)

1201.2345070838928


In [33]:
# load empty graph
g1 = Graph()

# parse processed GO (with RO object properties, and equivalent properties mapped) and protein instances of new equivalent classes
g1.parse( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/processed_ontology_human.xml', format = 'xml')
g1.parse( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/protein_instances_rdf.xml', format = 'xml')
g1.parse( '/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/equivalent_class.xml', format = 'xml')

# save modified protein ontology
g1.serialize(destination='/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Ontology_Data/preprocessed_all_data_xml.xml', format='xml')